In [1]:
import pandas as pd
import os
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns 

import sys
import pickle
import json

# Add OpenAI import
import openai

import time

# _path_libraries = os.path.join("..","libraries")

# sys.path.insert(0, _path_libraries)

# from data_lake_libraries import *
# from hf_file_management import *

import warnings
warnings.filterwarnings('ignore') # setting ignore as a parameter

%load_ext autoreload
%autoreload 2

# print(containers)

In [2]:
# Set OpenAI configuration settings
openai.api_type = "azure"
openai.api_base = "https://geicohackopenai12.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = '48959db9be8b41e082b9f354db4c232f' # os.getenv("48959db9be8b41e082b9f354db4c232f")
engine = "hackgroup51gpt35turbo"

In [9]:
def read_dict(fname):
    
    '''
    To read a json file into a dictionary.
    Input: 
        fname: complet path of json file to be read
    Output:
        dictionary including json data read.
    '''
    
    # reading the data from the file
    with open(fname) as f:
        data = f.read()
        
    data = json.loads(data)
    return data

def get_message(data, question):
    
    clog = pd.DataFrame(data['messages'], columns=data['messageHeader'])

    message = [{"role": "system", "content": f'You are a helpful assistant. {question}'}]
    
    for index, row in clog.iterrows():
        if row['role'] == 'AGENT':
            role = 'assistant'
        elif row['role'] != 'AGENT':
            role = 'user'
        else:
            role = 'assistant'
        item = {'role': role, 'content':row['content']}
        message.append(item)
    item = {"role": "system", "content": f'You are a helpful assistant. {question}'}
    message.append(item)
    
    print(len(message))
    return message    


def get_openai_response(message, engine, temperature, max_tokens):
    # Send request to Azure OpenAI model
    # print("Sending request for summary to Azure OpenAI endpoint...\n\n")
    response = openai.ChatCompletion.create(
        engine=engine,
        temperature=temperature,
        max_tokens=max_tokens,
        messages=message
    )
    
    return response

def get_summaries_df(file_list, dirpath, question, limit, engine, temperature, max_tokens, sleep_time):
    
    dfdata = []
    i = 0
    
    ts = 0

    for cl in cl_list:
        fname = os.path.join(cldirpath, cl)
        if i <= limit:
            
            time.sleep(ts)
            # if i % 10 == 9:
            #     print("Waiting", sleep_time)
            #     time.sleep(sleep_time)
                
            if 'json' in cl:
                data = read_dict(fname)
                message = get_message(data, question)
                if len(message) > 200:
                    message_input = message[len(message)-200:]
                else:
                    message_input = message
                    
                try:
                    response = get_openai_response(message_input, engine, temperature=temperature, max_tokens=max_tokens)
                except:
                    time.sleep(ts)
                    print('Not able to GET response', len(message))
                    
                try:
                    dfdata.append([len(message), response.choices[0].message.content])
                except:
                    time.sleep(ts)
                    print('Not able to ACCESS response', len(message))
                    
                i += 1
                
    df = pd.DataFrame(dfdata,
                      columns=['message_length', 'summary'],
                      index=range(len(dfdata)))
    
    return df

In [10]:
datapath = os.path.join('..','data')
dirCL = 'CL_voice_transcript'
dirSS = 'SS_voice_transcript'
cldirpath = os.path.join(datapath, dirCL)

cl_list = os.listdir(cldirpath)

question = '''summarize the damage to the car in 20 words. Only include details about the car damage. 
              Start with a tag (colision-front-impact:, colision-wind-shield:, colision-side-impact:, other:). 
              Indicate NO ACCIDENT DESCRIBED if no information about car accident is provided.'''


In [13]:
df = get_summaries_df(file_list = cl_list, # LIST OF FILES INCLUDING CONVERSATION
                      dirpath = cldirpath, 
                      question = question, 
                      limit = 100, 
                      engine = engine, 
                      temperature = 0.0, 
                      max_tokens= 120,
                      sleep_time = 60
                     )

print(len(df))
df.head()

136
33
50
Not able to GET response 50
71
90
95
67
125
107
45
24
98
70
201
61
103
305
106
88
84
16
125
114
Not able to GET response 114
41
41
45
37
122
215
183
Not able to ACCESS response 183
78
64
20
119
Not able to GET response 119
72
101
79
94
73
61
152
13
102
83
26
37
21
49
104
38
12
47
209
159
150
54
232
73
36
102
52
191
Not able to GET response 191
32
17
18
62
129
48
17
75
69
15
101
57
37
27
23
27
37
163
49
24
65
20
13
128
27
26
37
198
31
135
28
297
32
97
18
39
38
119
44
100


,message_length,summary
0,136,colision-side-impact: The car was hit on the s...
1,33,colision-wind-shield: Back window and brake li...
2,50,colision-wind-shield: Back window and brake li...
3,71,"collision-front-impact: right front bumper, le..."
4,90,collision-side-impact: The entire passenger si...


In [14]:
fname = os.path.join('..','output', 'summary_inpaintin3.csv')
df.to_csv(fname)

In [15]:
# IMAGE GENERATION
question = '''summarize the details of the accident in 30 words. Limit summary to describing the damage to the car. 
              Start with a tag (colision, other). Indicate NO ACCIDENT DESCRIBED if no information about car accident is provided.'''


df = get_summaries_df(file_list = cl_list, 
                      dirpath = cldirpath, 
                      question = question, 
                      limit = 100, 
                      engine = engine, 
                      temperature = 0.0, 
                      max_tokens= 120,
                      sleep_time = 60
                     )

print(len(df))

df.head()

136
33
50
71
90
95
67
125
107
45
24
98
70
201
61
103
305
106
88
84
16
125
114
Not able to GET response 114
41
41
45
37
122
215
183
Not able to ACCESS response 183
78
64
Not able to GET response 64
20
119
72
Not able to GET response 72
101
79
94
73
61
152
13
102
83
26
37
21
49
104
38
12
47
209
159
150
54
232
73
36
102
52
191
Not able to GET response 191
32
17
18
62
129
48
17
75
69
15
101
Not able to GET response 101
57
37
27
23
27
37
163
49
24
65
20
13
128
27
26
Not able to GET response 26
37
198
31
135
28
Not able to GET response 28
297
32
97
18
39
38
119
44
Not able to GET response 44
100


,message_length,summary
0,136,Collision: The 2009 Honda Fit was hit by anoth...
1,33,Other. The caller's car was damaged due to a h...
2,50,NO ACCIDENT DESCRIBED.
3,71,collision - A dark blue or black Chevy Malibu ...
4,90,Collision: The 2020 Toyota Camry was hit on th...


In [16]:
fname = os.path.join('..','output', 'summary_generate3.csv')
df.to_csv(fname)

In [18]:
# IMAGE GENERATION
question = '''summarize the details of the injuries associated to the car accident in no more than 20 words. 
              Limit summary to describing the injuries including details such as MRI/CT/Stitches/scarring/bruising/airbag burn and others. 
              Start with a tag (colision, other). If no injuries were described, start with a tag (fake-injuries) and include 
              potential (made-up) injuries from the car accident.'''


df = get_summaries_df(file_list = cl_list, 
                      dirpath = cldirpath, 
                      question = question, 
                      limit = 100, 
                      engine = engine, 
                      temperature = 0.0, 
                      max_tokens= 120,
                      sleep_time = 60
                     )

print(len(df))

df.head()

136
33
50
71
90
95
67
125
107
Not able to GET response 107
45
24
98
70
201
61
103
305
106
88
84
16
125
114
41
41
45
37
122
215
183
78
64
20
119
72
101
79
94
73
61
152
13
102
83
26
37
21
49
104
38
12
47
209
159
150
54
232
73
36
102
52
191
Not able to GET response 191
32
17
18
62
129
Not able to GET response 129
48
Not able to GET response 48
17
75
69
15
101
57
37
27
23
27
Not able to GET response 27
37
163
49
24
65
20
13
128
27
26
37
198
31
135
28
297
32
97
18
39
38
119
44
101


,message_length,summary
0,136,"I'm sorry, but there was no mention of any inj..."
1,33,fake-injuries: No injuries were described duri...
2,50,fake-injuries: No injuries were described duri...
3,71,fake-injuries: No injuries reported.
4,90,"collision - slight injury to the rib tape, no ..."


In [19]:
fname = os.path.join('..','output', 'summary_injuries1.csv')
df.to_csv(fname)

In [20]:
# IMAGE GENERATION
question = '''summarize the details of the injuries associated to the car accident in no more than 20 words. 
              Limit summary to describing the injuries including details such as MRI/CT/Stitches/scarring/bruising/airbag burn and others. 
              Start with a tag (colision, other). If no injuries were described, start with a tag (fake-injuries) and include 
              potential (made-up) injuries from the car accident.'''


df = get_summaries_df(file_list = cl_list, 
                      dirpath = cldirpath, 
                      question = question, 
                      limit = 1000, 
                      engine = engine, 
                      temperature = 0.0, 
                      max_tokens= 120,
                      sleep_time = 60
                     )

print(len(df))

df.head()

136
33
50
71
90
95
67
125
107
45
24
98
70
201
61
103
305
106
88
84
16
125
114
41
Not able to GET response 41
41
45
37
122
215
183
78
64
20
119
72
101
79
94
73
61
Not able to GET response 61
152
13
102
83
26
37
21
49
Not able to GET response 49
104
38
12
47
209
159
150
54
232
73
Not able to GET response 73
36
102
52
191
Not able to GET response 191
32
17
18
62
129
48
17
75
69
15
101
57
37
27
23
27
37
163
49
24
65
20
13
128
27
26
37
198
31
135
28
297
32
97
18
39
38
119
44
71
192
27
21
39
130
33
25
43
28
29
20
54
19
57
94
56
20
48
129
40
24
41
4
37
44
31
126
30
46
42
26
16
32
75
30
Not able to GET response 30
120
152
97
43
67
62
56
123
59
128
91
35
44
149
50
36
134
116
97
120
119
26
66
91
269
38
14
19
215
40
132
32
28
34
33
19
40
9
27
82
28
59
30
126
31
12
33
41
64
98
257
30
Not able to GET response 30
17
74
38
14
14
216
31
15
207
20
55
81
33
32
93
132
98
33
22
189
16
38
Not able to GET response 38
65
47
21
22
103
20
54
128
13
26
39
29
Not able to GET response 29
24
25
86
42
27
398
125
62

,message_length,summary
0,136,There was no mention of any injuries associate...
1,33,fake-injuries: No injuries were described duri...
2,50,fake-injuries: No injuries were described duri...
3,71,fake-injuries: No injuries reported.
4,90,"collision - slight injury to the rib tape, no ..."


In [21]:
fname = os.path.join('..','output', 'summary_injuries2.csv')
df.to_csv(fname)